In [2]:
#variables:
#primer dilutions:
stkprm = 100 #concentration of the stock primer you are adding
stkvol = 1 #the volume of stock primer you are adding
dilprm = 2.5 #this is the concentration in uM that you want your working dilution to be

#pcr reaction
# need to get this from the df##Numprimers = 4 #this is how many primers go in each pcr reaction.
primerconcentration = 0.1 #this is the concentration you want each primer to be in the pcr reaction
pcrvol = 25 #this is the total volume of your pcr reaction 
templatengs = .5 #this is the concentration of template you want in your pcr rxn in ng/uL

#template dilutions tells you what the temps need to be diluted to initially so that you can just add 1 uL of template to the pcr:
#need to fill in stock template values further down the script
diltemp = (templatengs)*(pcrvol)/1

total_volume = 25
Q5 = total_volume - (0.5*(total_volume)) #How much Q5 to add
DPNI = 1 #How much DPNI to add
DPwater = 19
cutsmart = 5

Date = '20210928'

#first import information from the j5 spreadsheet in order to perform appropriate steps
#import feather
#import pyarrow.feather as ft
import pandas
import numpy as np
import os

os.chdir("C:/Users/mkell/GitHub/opentrons/Golden_Gate/Part1_PCR_Mason")
os.getcwd()

pcr = pd.read_csv("pcr.csv")
assembly = pd.read_csv("assembly.csv")
combinations = pd.read_csv("combinations.csv")
oligo = pd.read_csv("oligo.csv")

In [3]:

os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Golden_Gate/Part1_PCR_Mason/")
os.getcwd()
oligos = pandas.read_csv('oligo.csv')
oligos


oligos['ID Number'] = oligos['ID Number'].astype(int)
oligos

if len(oligos.columns) < 9:
    oligos['well'] = ''
    oligos['stock primer concentration'] = ''
    oligos['volume of stock primer to add'] = ''
    oligos['concentration of diluted primer'] = ''
    oligos['volume of diluted primer'] = '' #this is a calculated value
    oligos['how much of the diluted primer is left'] = '' #also a calculated value
oligos

#custom 4x6 well plate dictionary. hardcoded specifically for the labware used. 
#this could easily be replace with another well specification dictionary

id2well = {}
id2well['0'] = 'A1'
id2well['1'] = 'A2'
id2well['2'] = 'A3'
id2well['3'] = 'A4'
id2well['4'] = 'A5'
id2well['5'] = 'A6'
id2well['6'] = 'B1'
id2well['7'] = 'B2'
id2well['8'] = 'B3'
id2well['9'] = 'B4'
id2well['10'] = 'B5'
id2well['11'] = 'B6'
id2well['12'] = 'C1'
id2well['13'] = 'C2'
id2well['14'] = 'C3'
id2well['15'] = 'C4'
id2well['16'] = 'C5'
id2well['17'] = 'C6'
id2well['18'] = 'D1'
id2well['19'] = 'D2'
id2well['20'] = 'D3'
id2well['21'] = 'D4'
id2well['22'] = 'D5'
id2well['23'] = 'D6'

for i, row in oligos.iterrows():
    oligos.loc[i,'well'] = id2well[str(i)] #this only works because the index matces the id number. id number is a floating value
    oligos.loc[i,'stock primer concentration'] = stkprm
    oligos.loc[i,'volume of stock primer to add'] = stkvol
    oligos.loc[i,'concentration of diluted primer'] = dilprm
    
for i, row in oligos.iterrows():
    oligos.loc[i,'volume of diluted primer'] = row['stock primer concentration']*row['volume of stock primer to add']/row['concentration of diluted primer']
    
oligos

#oligos.to_csv('output_'+Date+'_oligo_IVA.csv')


,ID Number,Name,Length,Tm,Tm (3' only),Cost,Sequence,Sequence (3' only),well,stock primer concentration,volume of stock primer to add,concentration of diluted primer,volume of diluted primer,how much of the diluted primer is left
0,0,oWL00250_(GFP_plus_link)_forward,42,65.027,54.766,4.2,CACACCAGGTCTCAAATGATGTCTAAAGGTGAAGAATTATTC,ATGTCTAAAGGTGAAGAATTATTC,A1,100,1,2.5,40.0,
1,1,oWL00251_(GFP_plus_link)_reverse,34,76.427,65.048,3.4,CACACCAGGTCTCAATCCCACCTGATCCGCCACC,CCCACCTGATCCGCCACC,A2,100,1,2.5,40.0,
2,2,oWL00252_(iaa12-prb)_forward,34,68.964,47.768,3.4,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGA,ATGTCTTCTCTTAGTAGA,A3,100,1,2.5,40.0,
3,3,oWL00253_(iaa12-prb)_reverse,36,74.704,61.651,3.6,CACACCAGGTCTCAAGAATCAGTGGTGGTGGTGGTG,TCAGTGGTGGTGGTGGTG,A4,100,1,2.5,40.0,


In [4]:
oligos['amount primer to add to frag amplification'] = pcrvol*primerconcentration/oligos['concentration of diluted primer']
oligos

,ID Number,Name,Length,Tm,Tm (3' only),Cost,Sequence,Sequence (3' only),well,stock primer concentration,volume of stock primer to add,concentration of diluted primer,volume of diluted primer,how much of the diluted primer is left,amount primer to add to frag amplification
0,0,oWL00250_(GFP_plus_link)_forward,42,65.027,54.766,4.2,CACACCAGGTCTCAAATGATGTCTAAAGGTGAAGAATTATTC,ATGTCTAAAGGTGAAGAATTATTC,A1,100,1,2.5,40.0,,1.0
1,1,oWL00251_(GFP_plus_link)_reverse,34,76.427,65.048,3.4,CACACCAGGTCTCAATCCCACCTGATCCGCCACC,CCCACCTGATCCGCCACC,A2,100,1,2.5,40.0,,1.0
2,2,oWL00252_(iaa12-prb)_forward,34,68.964,47.768,3.4,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGA,ATGTCTTCTCTTAGTAGA,A3,100,1,2.5,40.0,,1.0
3,3,oWL00253_(iaa12-prb)_reverse,36,74.704,61.651,3.6,CACACCAGGTCTCAAGAATCAGTGGTGGTGGTGGTG,TCAGTGGTGGTGGTGGTG,A4,100,1,2.5,40.0,,1.0


In [5]:
#read in assembly pieces as dataframe .... might not need this info
os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Golden_Gate/Part1_PCR_Mason/")
os.getcwd()
assembly = pandas.read_csv('assembly.csv')
assembly

#assembly.to_csv('output_'+Date+'_assembly_IVA.csv')

,Reaction ID Number,Reaction Type,Type ID Number,Part(s),Overhang with Previous,Overhang with Next,Relative Overhang Position,Sequence Length,Sequence
0,0,Digest Linearized,0,(pIWG),TTCT,AATG,-2,4789,TTCTGTCATGTAATTAGTTATGTCACGCTTACATTCACGCCCTCCC...
1,1,PCR,0,(GFP_plus_link),AATG,GGAT,0,793,CACACCAGGTCTCAAATGATGTCTAAAGGTGAAGAATTATTCACTG...
2,2,PCR,1,(iaa12-prb),GGAT,TTCT,2,1180,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...
3,3,PCR,2,(iaa12-pdar),GGAT,TTCT,2,1219,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...
4,4,PCR,3,(iaa14-prb),GGAT,TTCT,2,1138,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...
5,5,PCR,4,(iaa14-pdar),GGAT,TTCT,2,1177,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...


In [6]:
os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Golden_Gate/Part1_PCR_Mason/")
os.getcwd()
digests = pandas.read_csv('digests.csv')
digests
digest_conc=138 #automate digest concentration entry with inputs.py
digests['digest_conc']=digest_conc
digests
startnum = len(oligos['well'])
for i, row in digests.iterrows():
    digests.loc[i,'well'] = id2well[str(startnum+ i)]
digests

next_startnum = startnum+len(digests['well'])
digests

,ID Number,Sequence Source,Length,Sequence,digest_conc,well
0,0,pWL34_pIWG_ccdB,4789,TTCTGTCATGTAATTAGTTATGTCACGCTTACATTCACGCCCTCCC...,138,A5


In [26]:
for i, row in digests.iterrows():
    digests.loc[i,'amount of template to add'] = 1


for i, row in digests.iterrows():
   
        digests.loc[i,'concentration of template (ng/uL)'] = diltemp

digests['volume of dilute template prepared'] = digests['digest_conc']*stkvol/digests['concentration of template (ng/uL)']

digests['water to add']= digests['volume of dilute template prepared']-digests['amount of template to add']
digests

,ID Number,Sequence Source,Length,Sequence,digest_conc,well,amount of template to add,concentration of template (ng/uL),volume of dilute template prepared,water to add
0,0,pWL34_pIWG_ccdB,4789,TTCTGTCATGTAATTAGTTATGTCACGCTTACATTCACGCCCTCCC...,138,A5,1.0,12.5,11.04,10.04


In [8]:
#bumpback = 0

#for i, row in assmebly.iterrows():
#    if assembly['Reaction Type'] == 'Digest Linearized'
#        bumpback = bumpback + 1
#        assembly['']

#bumpback = 1

In [9]:
os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Golden_Gate/Part1_PCR_Mason/")
os.getcwd()
pcr = pandas.read_csv('pcr.csv')
pcr.columns = pcr.columns.str.replace("'","")
pcr

pcr[['Reaction ID Number','Forward Oligo ID Number','Reverse Oligo ID Number']] = pcr[['Reaction ID Number','Forward Oligo ID Number','Reverse Oligo ID Number']].astype(int)
pcr


#if bumpback > 0:
#    pcr.index = np.arange(1, len(pcr) + 1)
pcr

,Reaction ID Number,Primary Template,Alternate Template,Forward Oligo ID Number,Forward Oligo Name,Reverse Oligo ID Number,Reverse Oligo Name,Notes,Mean Oligo Tm,Delta Oligo Tm,Mean Oligo Tm (3 Only),Delta Oligo Tm (3Only),Length,Sequence
0,0,pWL233_pGP4G2_yE,NaN,0,oWL00250_(GFP_plus_link)_forward,1,oWL00251_(GFP_plus_link)_reverse,PCR,70.727,11.40,59.9070,10.282,793,CACACCAGGTCTCAAATGATGTCTAAAGGTGAAGAATTATTCACTG...
1,1,pwl248,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,54.7095,13.883,1180,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...
2,2,pwl277,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,54.7095,13.883,1219,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...
3,3,pwl282,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,54.7095,13.883,1138,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...
4,4,pwl281,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,54.7095,13.883,1177,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...


In [10]:
#here we create an object with each unique template from the
templates = pcr["Primary Template"]
unique_templates = templates.drop_duplicates(keep = 'first', inplace = False)
unique_templates

df = pandas.DataFrame(unique_templates)

df = df.reset_index()
df = df.drop('index', 1)

df['Template Concentration'] = ''
df

,Primary Template,Template Concentration
0,pWL233_pGP4G2_yE,
1,pwl248,
2,pwl277,
3,pwl282,
4,pwl281,


In [11]:
#enter template concentrations here
df['Template Concentration'] = [455,377,496,877,231] #all you have to do is input the template concentrations in the right order

df

startnum = len(oligos['well'])

for i, row in df.iterrows():
    df.loc[i,'template_well'] = id2well[str(next_startnum+i)]
    


for i, row in df.iterrows():
    df.loc[i,'amount of template to add'] = 1


for i, row in df.iterrows():
   
        df.loc[i,'concentration of template (ng/uL)'] = diltemp

df['volume of dilute template prepared'] = df['Template Concentration']*stkvol/df['concentration of template (ng/uL)']

df['water to add']= df['volume of dilute template prepared']-df['amount of template to add']


#df['volume of dilute template prepared'] = df['volume of dilute template prepared'].astype(float)
#df['concentration of template (ng/uL)'] = df['concentration of template (ng/uL)'].astype(float)
#df['amount of template to add'] = df['amount of template to add'].astype(float)

#df['water to add'] = df['water to add'].astype(int)
#df['concentration of template (ng/uL)'] = df['concentration of template (ng/uL)'].astype(int)
#df['volume of dilute template prepared'] = df['volume of dilute template prepared'].astype(int)
#df['amount of template to add'] = df['amount of template to add'].astype(int)

df['water to add'].astype(np.float32)
df['amount of template to add'].astype(np.float32)
df['concentration of template (ng/uL)'].astype(np.float32)
df['volume of dilute template prepared'].astype(np.float32)

#df['amount of template to add'] = pd.Series.astype(df['amount of template to add'], dtype=float)
#df['concentration of template (ng/uL)'] = pd.Series.astype(df['concentration of template (ng/uL)'], dtype=float)
#df['volume of dilute template prepared'] = pd.Series.astype(df['volume of dilute template prepared'], dtype=float,)
df

#if bumpback > 0:
#    df.index = np.arange(0+bumpback, len(df) + bumpback)
    
df
#df.to_csv('output_'+Date+'_templates_IVA.csv')

#df.dtypes

,Primary Template,Template Concentration,template_well,amount of template to add,concentration of template (ng/uL),volume of dilute template prepared,water to add
0,pWL233_pGP4G2_yE,455,A6,1.0,12.5,36.40,35.40
1,pwl248,377,B1,1.0,12.5,30.16,29.16
2,pwl277,496,B2,1.0,12.5,39.68,38.68
3,pwl282,877,B3,1.0,12.5,70.16,69.16
4,pwl281,231,B4,1.0,12.5,18.48,17.48


In [12]:
#this line of code integrates the template concentrations into the pcr df
pcr_plustemplates = pandas.merge(pcr,df,on='Primary Template')
pcr_plustemplates

if len(pcr_plustemplates.columns) == 17:

    for i, row in pcr_plustemplates.iterrows():
   
        pcr_plustemplates.loc[i,'concentration of template (ng/uL)'] = diltemp

pcr_plustemplates['volume of dilute template prepared'] = pcr_plustemplates['Template Concentration']*stkvol/pcr_plustemplates['concentration of template (ng/uL)']

pcr_plustemplates

,Reaction ID Number,Primary Template,Alternate Template,Forward Oligo ID Number,Forward Oligo Name,Reverse Oligo ID Number,Reverse Oligo Name,Notes,Mean Oligo Tm,Delta Oligo Tm,Mean Oligo Tm (3 Only),Delta Oligo Tm (3Only),Length,Sequence,Template Concentration,template_well,amount of template to add,concentration of template (ng/uL),volume of dilute template prepared,water to add
0,0,pWL233_pGP4G2_yE,NaN,0,oWL00250_(GFP_plus_link)_forward,1,oWL00251_(GFP_plus_link)_reverse,PCR,70.727,11.40,59.9070,10.282,793,CACACCAGGTCTCAAATGATGTCTAAAGGTGAAGAATTATTCACTG...,455,A6,1.0,12.5,36.40,35.40
1,1,pwl248,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,54.7095,13.883,1180,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...,377,B1,1.0,12.5,30.16,29.16
2,2,pwl277,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,54.7095,13.883,1219,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...,496,B2,1.0,12.5,39.68,38.68
3,3,pwl282,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,54.7095,13.883,1138,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...,877,B3,1.0,12.5,70.16,69.16
4,4,pwl281,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,54.7095,13.883,1177,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...,231,B4,1.0,12.5,18.48,17.48


In [13]:
wellinfo = oligos[['ID Number','well']]
wellinfo

wellinfo = wellinfo.rename(columns={'ID Number':'Forward Oligo ID Number'})
pcr_plustemplates = pcr_plustemplates.merge(wellinfo, on= 'Forward Oligo ID Number')
wellinfo = wellinfo.rename(columns={'Forward Oligo ID Number':'Reverse Oligo ID Number','well':'well2'})
pcr_plustemplates = pcr_plustemplates.merge(wellinfo, on= 'Reverse Oligo ID Number')
pcr_plustemplates

#pcr_plustemplates.to_csv('output_'+Date+'_pcr_IVA.csv')

,Reaction ID Number,Primary Template,Alternate Template,Forward Oligo ID Number,Forward Oligo Name,Reverse Oligo ID Number,Reverse Oligo Name,Notes,Mean Oligo Tm,Delta Oligo Tm,...,Length,Sequence,Template Concentration,template_well,amount of template to add,concentration of template (ng/uL),volume of dilute template prepared,water to add,well,well2
0,0,pWL233_pGP4G2_yE,NaN,0,oWL00250_(GFP_plus_link)_forward,1,oWL00251_(GFP_plus_link)_reverse,PCR,70.727,11.40,...,793,CACACCAGGTCTCAAATGATGTCTAAAGGTGAAGAATTATTCACTG...,455,A6,1.0,12.5,36.40,35.40,A1,A2
1,1,pwl248,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,...,1180,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...,377,B1,1.0,12.5,30.16,29.16,A3,A4
2,2,pwl277,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,...,1219,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...,496,B2,1.0,12.5,39.68,38.68,A3,A4
3,3,pwl282,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,...,1138,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...,877,B3,1.0,12.5,70.16,69.16,A3,A4
4,4,pwl281,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,...,1177,CACACCAGGTCTCAGGATGTCTTCTCTTAGTAGAGAGCTCGTTTTC...,231,B4,1.0,12.5,18.48,17.48,A3,A4


In [14]:
pcr_plustemplates['total_water_toadd'] = total_volume-Q5-1-1-1
pcr_plustemplates


for i, row in pcr_plustemplates.iterrows():
    pcr_plustemplates.loc[i,'frag_pcr_tube'] = id2well[str(i)]
pcr_plustemplates

,Reaction ID Number,Primary Template,Alternate Template,Forward Oligo ID Number,Forward Oligo Name,Reverse Oligo ID Number,Reverse Oligo Name,Notes,Mean Oligo Tm,Delta Oligo Tm,...,Template Concentration,template_well,amount of template to add,concentration of template (ng/uL),volume of dilute template prepared,water to add,well,well2,total_water_toadd,frag_pcr_tube
0,0,pWL233_pGP4G2_yE,NaN,0,oWL00250_(GFP_plus_link)_forward,1,oWL00251_(GFP_plus_link)_reverse,PCR,70.727,11.40,...,455,A6,1.0,12.5,36.40,35.40,A1,A2,9.5,A1
1,1,pwl248,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,...,377,B1,1.0,12.5,30.16,29.16,A3,A4,9.5,A2
2,2,pwl277,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,...,496,B2,1.0,12.5,39.68,38.68,A3,A4,9.5,A3
3,3,pwl282,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,...,877,B3,1.0,12.5,70.16,69.16,A3,A4,9.5,A4
4,4,pwl281,NaN,2,oWL00252_(iaa12-prb)_forward,3,oWL00253_(iaa12-prb)_reverse,PCR,71.834,5.74,...,231,B4,1.0,12.5,18.48,17.48,A3,A4,9.5,A5


In [15]:
os.chdir("C:/Users/jonbr/Documents/GitHub/opentrons/Golden_Gate/Part1_PCR_Mason/")
os.getcwd()
combinations = pandas.read_csv('combinations.csv')
combinations



,ID Number,Name,Assembly Method,Part(s) Bin 0,Assembly Piece ID Number Bin 0,Part(s) Bin 1,Assembly Piece ID Number Bin 1,Part(s) Bin 2,Assembly Piece ID Number Bin 2
0,0,pmas00001,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa12-prb),2
1,1,pmas00002,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa12-pdar),3
2,2,pmas00003,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa14-prb),4
3,3,pmas00004,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa14-pdar),5


In [25]:

pieces = [columns for columns in combinations if columns.startswith('Assembly Piece ID Number Bin ')]
frame = combinations[pieces]
#frame2 = frame.transpose()
frame

frame += startnum
frame
frame= frame.values.astype(str)
frame = pandas.DataFrame(frame)
frame
result_1 = frame.replace(id2well)
result_1

combinations_plustemplocs = pandas.concat([combinations, result_1], axis=1)
combinations_plustemplocs

,ID Number,Name,Assembly Method,Part(s) Bin 0,Assembly Piece ID Number Bin 0,Part(s) Bin 1,Assembly Piece ID Number Bin 1,Part(s) Bin 2,Assembly Piece ID Number Bin 2,0,1,2
0,0,pmas00001,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa12-prb),2,A5,A6,B1
1,1,pmas00002,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa12-pdar),3,A5,A6,B2
2,2,pmas00003,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa14-prb),4,A5,A6,B3
3,3,pmas00004,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa14-pdar),5,A5,A6,B4


In [21]:
combinations[['ID Number','Assembly Piece ID Number Bin 0','Assembly Piece ID Number Bin 1','Assembly Piece ID Number Bin 2']] = combinations[['ID Number','Assembly Piece ID Number Bin 0','Assembly Piece ID Number Bin 1','Assembly Piece ID Number Bin 2']].astype(int)
combinations

primerlocations = pcr_plustemplates[['Reaction ID Number','well','well2']]
primerlocations


,Reaction ID Number,well,well2
0,0,A1,A2
1,1,A3,A4
2,2,A3,A4
3,3,A3,A4
4,4,A3,A4


if assembly.loc[0].at['Reaction Type'] == 'PCR':
    
    
    
    
    primerlocations = primerlocations.rename(columns={'Reaction ID Number':'Assembly Piece ID Number Bin 0'})
    combinations = combinations.merge(primerlocations, on= 'Assembly Piece ID Number Bin 0')
    
    primerlocations = primerlocations.rename(columns={'Assembly Piece ID Number Bin 0':'Assembly Piece ID Number Bin 1'})
    combinations = combinations.merge(primerlocations, on= 'Assembly Piece ID Number Bin 1')

    primerlocations = primerlocations.rename(columns={'Assembly Piece ID Number Bin 1':'Assembly Piece ID Number Bin 2'})
    combinations = combinations.merge(primerlocations, on= 'Assembly Piece ID Number Bin 2')



    combinations['primer concentrations'] = oligos['concentration of diluted primer']
    combinations['amount primer to add to IVA'] = pcrvol*primerconcentration/combinations['primer concentrations']

    templateinformation = pcr_plustemplates[['Reaction ID Number','template_well']]

    templateinformation = templateinformation.rename(columns={'Reaction ID Number':'Assembly Piece ID Number Bin 0'})
    combinations = combinations.merge(templateinformation, on= 'Assembly Piece ID Number Bin 0')

    templateinformation = templateinformation.rename(columns={'Assembly Piece ID Number Bin 0':'Assembly Piece ID Number Bin 1'})
    combinations = combinations.merge(templateinformation, on= 'Assembly Piece ID Number Bin 1')

    templateinformation = templateinformation.rename(columns={'Assembly Piece ID Number Bin 1':'Assembly Piece ID Number Bin 2'})
    combinations = combinations.merge(templateinformation, on= 'Assembly Piece ID Number Bin 2')

    combinations['template concentrations'] = pcr_plustemplates['concentration of template (ng/uL)']
    combinations['amount templates to add'] = pcrvol*templatengs/combinations['template concentrations']

combinations

In [63]:
#if assembly.loc[0].at['Reaction Type'] == 'Digest Linearized':




primerlocations['Reaction ID Number'] = primerlocations['Reaction ID Number'] + 1
primerlocations

primerlocations = primerlocations.rename(columns={'Reaction ID Number':'Assembly Piece ID Number Bin 1'})
primerlocations
combinations = combinations.merge(primerlocations, on= 'Assembly Piece ID Number Bin 1')
primerlocations = primerlocations.rename(columns={'Assembly Piece ID Number Bin 1':'Assembly Piece ID Number Bin 2'})
combinations = combinations.merge(primerlocations, on= 'Assembly Piece ID Number Bin 2')
combinations


<ipython-input-63-6864fb2083f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primerlocations['Reaction ID Number'] = primerlocations['Reaction ID Number'] + 1


,ID Number,Name,Assembly Method,Part(s) Bin 0,Assembly Piece ID Number Bin 0,Part(s) Bin 1,Assembly Piece ID Number Bin 1,Part(s) Bin 2,Assembly Piece ID Number Bin 2,well_x,well2_x,well_y,well2_y
0,0,pmas00001,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa12-prb),2,A1,A2,A3,A4
1,1,pmas00002,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa12-pdar),3,A1,A2,A3,A4
2,2,pmas00003,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa14-prb),4,A1,A2,A3,A4
3,3,pmas00004,Golden-gate,(pIWG),0,(GFP_plus_link),1,(iaa14-pdar),5,A1,A2,A3,A4


In [ ]:


combinations['primer concentrations'] = oligos['concentration of diluted primer']
combinations['amount primer to add to IVA'] = pcrvol*primerconcentration/combinations['primer concentrations']

templateinformation = pcr_plustemplates[['Reaction ID Number','template_well']]

templateinformation = templateinformation.rename(columns={'Reaction ID Number':'Assembly Piece ID Number Bin 0'})
combinations = combinations.merge(templateinformation, on= 'Assembly Piece ID Number Bin 0')

templateinformation = templateinformation.rename(columns={'Assembly Piece ID Number Bin 0':'Assembly Piece ID Number Bin 1'})
combinations = combinations.merge(templateinformation, on= 'Assembly Piece ID Number Bin 1')

templateinformation = templateinformation.rename(columns={'Assembly Piece ID Number Bin 1':'Assembly Piece ID Number Bin 2'})
combinations = combinations.merge(templateinformation, on= 'Assembly Piece ID Number Bin 2')

combinations['template concentrations'] = pcr_plustemplates['concentration of template (ng/uL)']
combinations['amount templates to add'] = pcrvol*templatengs/combinations['template concentrations']
combinations

In [26]:
filter_col = [col for col in combinations if col.startswith('well')] #here we calculate how much water to add
filter_col
primersadded = len(filter_col)

primersadded

36

In [ ]:
temp_col = [col for col in combinations if col.startswith('template_')] #here we calculate how much water to add
temp_col
tempsadded = len(temp_col)


combinations['water to add'] = (pcrvol-primersadded- tempsadded - Q5)


combinations

# id2wellpcr = {}
# id2wellpcr['0'] = 'A1'
# id2wellpcr['1'] = 'A2'
# id2wellpcr['2'] = 'A3'
# id2wellpcr['3'] = 'A4'
# id2wellpcr['4'] = 'A5'
# id2wellpcr['5'] = 'A6'
# id2wellpcr['6'] = 'A7'
# id2wellpcr['7'] = 'A8'
# id2wellpcr['8'] = 'A9'
# id2wellpcr['9'] = 'A10'
# id2wellpcr['10'] = 'A11'
# id2wellpcr['11'] = 'A12'

#Have to adjust this dictionary based on which pcr well is available.
#theoretically, we need 96 if statements with a variation of the dictionary starting at each well then you could
#just imput which well to start with 
id2wellpcr = {}
id2wellpcr['0'] = 'A3'
id2wellpcr['1'] = 'A4'
id2wellpcr['2'] = 'A5'
id2wellpcr['3'] = 'A6'
id2wellpcr['4'] = 'A7'
id2wellpcr['5'] = 'A8'
id2wellpcr['6'] = 'A9'
id2wellpcr['7'] = 'A10'
id2wellpcr['8'] = 'A11'
id2wellpcr['9'] = 'A12'
id2wellpcr['10'] = 'B1'
id2wellpcr['11'] = 'B2'

combinations['pcrwell']=combinations['ID Number']
for i, row in combinations.iterrows():
    combinations.loc[i,'pcrwell'] = id2wellpcr[str(i)]
combinations

#combinations.to_csv('output_'+Date+'_combination_IVA.csv')


In [ ]:
from opentrons import protocol_api

#Metadata is a dictionary of data that is read by the server and returned to the opentrons app. 
#give yourself credit. you are required to specify the 'apiLevel' herefrom opentrons import protocol_api
metadata = {
    'protocolName': 'ARF7 Deletions Protocol',
    'author': 'John Bryant <jbryant2@vt.edu>',
    'description': 'Protocol for performing PCR reactions and Plasmid assembly for TIR1 and AFB mutants',
    'apiLevel': '2.10'
}
print(metadata)




def run(protocol: protocol_api.ProtocolContext): #for actually running the script in the robot
#will have to indent everything to be defined by the run function

#from opentrons import simulate
#protocol = simulate.get_protocol_api('2.2')
    
#labware:
    tiprack1 = protocol.load_labware('opentrons_96_tiprack_300ul', '9')
    tiprack2 = protocol.load_labware('opentrons_96_tiprack_300ul','6')
    tiprack3 = protocol.load_labware("opentrons_96_tiprack_10ul", '5')
#tuberack1 = protocol.load_labware('opentrons_24_tuberack_generic_2ml_screwcap','1') #holds stock primers and templates
    watertuberack = protocol.load_labware('opentrons_10_tuberack_falcon_4x50ml_6x15ml_conical','3') #holds molec bio grad H2O
    tuberack2 = protocol.load_labware('opentrons_24_tuberack_nest_1.5ml_snapcap','2') # holds dilute primers and templates
    
    tc_mod = protocol.load_module('Thermocycler Module')
    pcrplate = tc_mod.load_labware('nest_96_wellplate_100ul_pcr_full_skirt')
    temp_module = protocol.load_module('temperature module', 1)
    cold_tuberack = temp_module.load_labware('opentrons_24_aluminumblock_generic_2ml_screwcap', label='Temperature-Controlled Tubes')
    temp_module.set_temperature(4)
    print(temp_module.temperature)
    tc_mod.open_lid()

#########Some notes:    
#specify the order of stock primers and template in tuberack1 here:
#good place to add the pop-up window
#A1-D3 = stock primers
#D4-D5 = stock templates
#stock tubes and dilution tubes need to be set up in the same order
#as of now Q5 is in 
    
#pipettes
    right_pipette = protocol.load_instrument('p300_single','right',tip_racks=[tiprack1, tiprack2])
    left_pipette = protocol.load_instrument('p10_single','left',tip_racks = [tiprack3])
    
##################################COMMANDS####################################
    
#add water to template dilution tubes. ***df is the template description dataframe
#Since we are just moving water I will use the same pipette tip to save plastic
    right_pipette.pick_up_tip()
    for i, row in df.iterrows():
        right_pipette.aspirate(volume = df.loc[i].at['water to add'], location = watertuberack['A1'], rate=2.0) #total vol dilute template - vol stock template to add
        right_pipette.dispense(df.loc[i].at['water to add'], tuberack2[df.loc[i].at['template_well']], rate=2.0)
        right_pipette.blow_out()
#digestions water
    for i, row in digests.iterrows():
        right_pipette.aspirate(volume = digests.loc[i].at['water to add'], location = watertuberack['A1'], rate=2.0) #total vol dilute template - vol stock template to add
        right_pipette.dispense(digests.loc[i].at['water to add'], tuberack2[digests.loc[i].at['well']], rate=2.0)
        right_pipette.blow_out()

#add water to primer dilution tubes
    for i, row in oligos.iterrows():
        right_pipette.aspirate(oligos.loc[i].at['volume of diluted primer']-oligos.loc[i].at['volume of stock primer to add'], watertuberack['A1'], rate=2.0) #need to put 39uL of water into each dilution tube for primers,) #we need to find better way to loop through these commands
        right_pipette.dispense(oligos.loc[i].at['volume of diluted primer']-oligos.loc[i].at['volume of stock primer to add'], tuberack2[oligos.loc[i].at['well']], rate=2.0)
        right_pipette.blow_out()
    right_pipette.drop_tip()    
    
#add stock templates to dilution tubes
    for i, row in df.iterrows():
        left_pipette.pick_up_tip()
        left_pipette.aspirate(df.loc[i].at['amount of template to add'], cold_tuberack[df.loc[i].at['template_well']], rate=2.0) #dilution well corresponds to stock well
        left_pipette.dispense(df.loc[i].at['amount of template to add'], tuberack2[df.loc[i].at['template_well']], rate=2.0) #makes a 12.5ng/uL template
        left_pipette.blow_out()
        left_pipette.drop_tip()

#add stock templates for digests:
    for i, row in digests.iterrows():
        left_pipette.pick_up_tip()
        left_pipette.aspirate(digests.loc[i].at['amount of template to add'], cold_tuberack[digests.loc[i].at['well']], rate=2.0) #dilution well corresponds to stock well
        left_pipette.dispense(digests.loc[i].at['amount of template to add'], tuberack2[digests.loc[i].at['well']], rate=2.0) #makes a 12.5ng/uL template
        left_pipette.blow_out()
        left_pipette.drop_tip()
    
#add stock primers to dilution tube
    for i, row in oligos.iterrows():
        left_pipette.pick_up_tip() #add in an iterrows function
        left_pipette.aspirate(oligos.loc[i].at['volume of stock primer to add'], cold_tuberack[oligos.loc[i].at['well']], rate=2.0)
        left_pipette.dispense(oligos.loc[i].at['volume of stock primer to add'], tuberack2[oligos.loc[i].at['well']], rate=2.0)
        left_pipette.blow_out()
        left_pipette.drop_tip()
    
#mix contents with pipette tip (reps, max volume, location) for templates and primers
    for i, row in df.iterrows():
        right_pipette.pick_up_tip()
        right_pipette.mix(3,df.loc[i].at['water to add'],tuberack2[df.loc[i].at['template_well']])
        right_pipette.blow_out()
        right_pipette.drop_tip()

    for i, row in digests.iterrows():
        right_pipette.pick_up_tip()
        right_pipette.mix(3,digests.loc[i].at['water to add'],tuberack2[digests.loc[i].at['well']])
        right_pipette.blow_out()
        right_pipette.drop_tip()
        
    for i, row in oligos.iterrows():
        right_pipette.pick_up_tip()
        right_pipette.mix(3,oligos.loc[i].at['volume of diluted primer']-oligos.loc[i].at['volume of stock primer to add'],tuberack2[oligos.loc[i].at['well']])
        right_pipette.blow_out()
        right_pipette.drop_tip()

#robot pauses so user can take out stock primers and put in DNPNI
    protocol.pause('Take all stock primers and templates out. Add DPNI to A1, water to A2, and cutsmart to A3. Then proceed')
    
#now mix dilute primers, dilute templates, Q5, and water in pcr tube within thermocycler
    tc_mod.open_lid()
    
#Do the Digestion











#add water first
    for i, row in pcr_plustemplates.iterrows():
        
        right_pipette.pick_up_tip()
        right_pipette.aspirate(pcr_plustemplates.loc[i].at['total_water_toadd'], watertuberack['A1'], rate=2.0) #need to write a function to add up all volumes that are being added and figure out how much water to add in automated way
        right_pipette.dispense(pcr_plustemplates.loc[i].at['total_water_toadd'], pcrplate[pcr_plustemplates.loc[i].at['frag_pcr_tube']], rate=2.0)
        right_pipette.blow_out()
        right_pipette.drop_tip()
    
#add 1uL of BOTH (not each) primers
    for i, row in oligos.iterrows():
        left_pipette.pick_up_tip()
        left_pipette.aspirate(oligos.loc[i].at['amount primer to add to frag amplification'], tuberack2[pcr_plustemplates.loc[i].at[well]], rate=2.0)
        left_pipette.dispense(oligos.loc[i].at['amount primer to add to frag amplification'], pcrplate[pcr_plustemplates.loc[i].at['frag_pcr_tube']], rate=2.0)
        left_pipette.blow_out()            
        
        left_pipette.drop_tip()
        left_pipette.pick_up_tip()
        left_pipette.aspirate(oligos.loc[i].at['amount primer to add to frag amplification'], tuberack2[pcr_plustemplates.loc[i].at[well2]], rate=2.0)            
        left_pipette.dispense(oligos.loc[i].at['amount primer to add to frag amplification'], pcrplate[pcr_plustemplates.loc[i].at['frag_pcr_tube']], rate=2.0)
        left_pipette.blow_out()
        left_pipette.drop_tip()
    
#add 1uL of each template
    for i, row in combinations.iterrows():
        left_pipette.pick_up_tip()
        left_pipette.aspirate(pcr_plustemplates.loc[i].at['amount of template to add'], tuberack2[pcr_plustemplates.loc[i].at['template_well']], rate=2.0)
        left_pipette.dispense(pcr_plustemplates.loc[i].at['amount of template to add'], pcrplate[pcr_plustemplates.loc[i].at['frag_pcr_tube']], rate=2.0)
        left_pipette.blow_out()
        left_pipette.drop_tip()
    
#add Q5 to each reaction
#keep Q5 in tuberack1['D6']                                            
    for i, row in combinations.iterrows():
        right_pipette.pick_up_tip()
        right_pipette.aspirate(Q5, cold_tuberack['D6'], rate=2.0)
        right_pipette.aspirate(Q5, pcrplate[pcr_plustemplates.loc[i].at['frag_pcr_tube']], rate=2.0)
        right_pipette.blow_out()
        right_pipette.drop_tip()

#mix up
    for i, row in combinations.iterrows():
        right_pipette.pick_up_tip()
        right_pipette.mix(3,Q5+3,pcrplate[pcr_plustemplates.loc[i].at['frag_pcr_tube']])
        right_pipette.blow_out()
        right_pipette.drop_tip()
    
#Now run thermocycler to amplify DNA
    
#these parameters can be altered for different pcr reactionsabs
#should automate calculation of the parameters from j5 spreadsheets.
#maybe use the median annealing temperature in the spreadsheet
    
    tc_mod.set_lid_temperature(temperature = 105)
    tc_mod.set_block_temperature(98, hold_time_seconds=30, block_max_volume=25)
    profile = [
        {'temperature': 98, 'hold_time_seconds': 10},
        {'temperature': round(Annealing_and_extension.loc[1].at['Annealing temp'],1), 'hold_time_seconds': 30},
        {'temperature': 72, 'hold_time_seconds': round(Annealing_and_extension.loc[0].at['extension time (seconds)'],1)}] #should automate calculation of annealing temp based on spreadsheet
    tc_mod.execute_profile(steps=profile, repetitions=34, block_max_volume=25)
    tc_mod.set_block_temperature(72, hold_time_minutes=5, block_max_volume=25)
    tc_mod.set_block_temperature(4)
    tc_mod.open_lid()

#Now add DPNI for digestion

    for i, row in combinations.iterrows():
        right_pipette.pick_up_tip()
        right_pipette.aspirate(DPwater, cold_tuberack['A2'], rate=2.0)
        right_pipette.aspirate(DPwater, pcrplate[combinations.loc[i].at['pcrwell']], rate=2.0)
        right_pipette.drop_tip()

    for i, row in combinations.iterrows():
        left_pipette.pick_up_tip()
        left_pipette.aspirate(cutsmart, cold_tuberack['A3'], rate=2.0)
        left_pipette.aspirate(cutsmart, pcrplate[combinations.loc[i].at['pcrwell']], rate=2.0)
        left_pipette.drop_tip() 

    for i, row in combinations.iterrows():
        left_pipette.pick_up_tip()
        left_pipette.aspirate(DPNI, cold_tuberack['A1'], rate=2.0)
        left_pipette.aspirate(DPNI, pcrplate[combinations.loc[i].at['pcrwell']], rate=2.0)
        left_pipette.drop_tip()

    tc_mod.close_lid()
    tc_mod.set_block_temperature(37, hold_time_minutes=15, block_max_volume=50)
    tc_mod.set_block_temperature(80, hold_time_minutes=20, block_max_volume=50)
    tc_mod.set_block_temperature(4)
    tc_mod.deactivate_lid()
    protocol.pause('hold until time to grab tubes')
    
    tc_mod.open_lid()

    print('all done')

In [ ]:

os.chdir("/data/user_storage/Restriction/20210927_pme1")
os.getcwd()

plasmid = pandas.read_csv("20210927_pme1.csv",engine = 'python', encoding='utf-8-sig') #exception made here, don't repeat this file structure

plasmid = pandas.DataFrame(plasmid)

plasmid['Buffer'] = float('5')
plasmid['PME1'] = float('1')
plasmid['Volume of Plasmid'] = ''
plasmid['Volume of Water'] = ''

plasmid['Volume of Plasmid'] = (1/(plasmid['Concentration'])) * 1000 * 1
plasmid['Volume of Water'] = 44 - plasmid['Volume of Plasmid'] - plasmid['Buffer'] - plasmid['PME1']

plasmid['total volume'] = float('44')


#plasmid templates arranged in an "L" formation
row2well= {}
row2well['0'] = 'A2'
row2well['1'] = 'B2'
row2well['2'] = 'C2'
row2well['3'] = 'D2'
row2well['4'] = 'D3'
row2well['5'] = 'D4'
row2well['6'] = 'D5'
row2well['7'] = 'D6'

plasmid['Plasmid Location'] = ''

for i, row in plasmid.iterrows():
    plasmid.loc[i,'Plasmid Location'] = row2well[str(i)]

#The pcr tube destination will change every time you use a new row of the 96 well pcr plate
row2tube= {}
#row2tube['0'] = 'B1'
#row2tube['1'] = 'B2'
#row2tube['2'] = 'B3'
#row2tube['3'] = 'B4'
#row2tube['4'] = 'B5'
#row2tube['5'] = 'B6'
#row2tube['6'] = 'B7'
#row2tube['7'] = 'B8'
# row2tube['0'] = 'C1'
# row2tube['1'] = 'C2'
# row2tube['2'] = 'C3'
# row2tube['3'] = 'C4'
# row2tube['4'] = 'C5'
# row2tube['5'] = 'C6'
# row2tube['6'] = 'C7'
# row2tube['7'] = 'C8'
# row2tube['0'] = 'F1'
# row2tube['1'] = 'F2'
# row2tube['2'] = 'F3'
# row2tube['3'] = 'F4'
# row2tube['4'] = 'F5'
# row2tube['5'] = 'F6'
# row2tube['6'] = 'F7'
# row2tube['7'] = 'F8'
# row2tube['8'] = 'F9'
# row2tube['0'] = 'A3'
# row2tube['1'] = 'A4'
# row2tube['2'] = 'A5'
# row2tube['3'] = 'A6'
# row2tube['4'] = 'A7'
# row2tube['5'] = 'A8'
# row2tube['6'] = 'A9'
# row2tube['7'] = 'A10'
# row2tube['8'] = 'A11'

row2tube['0'] = 'A11'
row2tube['1'] = 'A12'
row2tube['2'] = 'B1'
row2tube['3'] = 'B2'

plasmid['Digestion Tube'] = ''

for i, row in plasmid.iterrows():
    plasmid.loc[i,'Digestion Tube'] = row2tube[str(i)]

pcr2final= {}
pcr2final['0'] = 'A1'
pcr2final['1'] = 'A2'
pcr2final['2'] = 'A3'
pcr2final['3'] = 'A4'
pcr2final['4'] = 'A5'
pcr2final['5'] = 'A6'
pcr2final['6'] = 'B1'
pcr2final['7'] = 'B2'
pcr2final['8'] = 'B3'

plasmid['final tube'] = ''

for i, row in plasmid.iterrows():
    plasmid.loc[i,'final tube'] = pcr2final[str(i)]



from opentrons import protocol_api

#Metadata is a dictionary of data that is read by the server and returned to the opentrons app. 
#give yourself credit. you are required to specify the 'apiLevel' herefrom opentrons import protocol_api
metadata = {
    'protocolName': 'PME1 Digestion',
    'author': 'Mason Kellinger <masonkellinger@vt.edu> and John Bryant <jbryant2@vt.edu',
    'description': 'Protocol for digesting plasmids that have PME1 cut sites',
    'apiLevel': '2.10'
}
#print(metadata)

#################################################################################
#Commands
#############################################################################
def run(protocol: protocol_api.ProtocolContext):

     ## Loading labware
    thermo = protocol.load_module('Thermocycler Module')
    pcr = thermo.load_labware('nest_96_wellplate_100ul_pcr_full_skirt')
        
    tiprack1 = protocol.load_labware("opentrons_96_tiprack_10ul",5)
    tiprack2 = protocol.load_labware("opentrons_96_tiprack_300ul",2)
    finalprodtubes = protocol.load_labware("opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap",3)
    #tuberack = protocol.load_labware("opentrons_24_tuberack_nest_2ml_snapcap",4)
    temp_module = protocol.load_module('temperature module', 1)
    cold_tuberack = temp_module.load_labware('opentrons_24_aluminumblock_nest_1.5ml_snapcap', label='Temperature-Controlled Tubes')
    temp_module.set_temperature(4)
    print(temp_module.temperature)
    
 ## Loading tools
    pipette_right = protocol.load_instrument('p300_single_gen2','right',tip_racks = [tiprack2])
    pipette_left = protocol.load_instrument('p10_single','left',tip_racks = [tiprack1])
## Commands

    thermo.open_lid()

    protocol.pause('Prepare Tubes')

# pick up water -> dispense into pcr tube within thermocycler -> get rid of tip
    pipette_right.pick_up_tip()
    for i, row in plasmid.iterrows():
        pipette_right.aspirate(plasmid.loc[i].at['Volume of Water'],cold_tuberack['A1'],2)
        pipette_right.dispense(plasmid.loc[i].at['Volume of Water'],pcr[plasmid.loc[i].at['Digestion Tube']],2)
        pipette_right.blow_out()

    pipette_right.drop_tip()

# pick up plasmid  -> dispense into pcr tube -> get rid of tip  no blow out because aeresol
    
    for i, row in plasmid.iterrows():
        if plasmid.loc[i].at['Volume of Plasmid'] < 10:
            pipette_left.pick_up_tip()
            pipette_left.aspirate(plasmid.loc[i].at['Volume of Plasmid'],cold_tuberack[plasmid.loc[i].at['Plasmid Location']],2) #location of plasmid
            pipette_left.dispense(plasmid.loc[i].at['Volume of Plasmid'],pcr[plasmid.loc[i].at['Digestion Tube']],2)
            pipette_left.mix(3,plasmid.loc[i].at['Volume of Plasmid'],pcr[plasmid.loc[i].at['Digestion Tube']])
            pipette_left.blow_out()
            pipette_left.drop_tip()

        if plasmid.loc[i].at['Volume of Plasmid'] > 10:
            pipette_right.pick_up_tip()
            pipette_right.aspirate(plasmid.loc[i].at['Volume of Plasmid'],cold_tuberack[plasmid.loc[i].at['Plasmid Location']],2) #location of plasmid
            pipette_right.dispense(plasmid.loc[i].at['Volume of Plasmid'],pcr[plasmid.loc[i].at['Digestion Tube']],2)
            pipette_right.mix(3,plasmid.loc[i].at['Volume of Plasmid'],pcr[plasmid.loc[i].at['Digestion Tube']])
            pipette_right.blow_out()
            pipette_right.drop_tip()

# pick up buffer  -> dispense into pcr tube -> get rid of tip

    for i, row in plasmid.iterrows():
        pipette_left.pick_up_tip()
        pipette_left.aspirate(plasmid.loc[i].at['Buffer'],cold_tuberack['A3'],2)
        pipette_left.dispense(plasmid.loc[i].at['Buffer'],pcr[plasmid.loc[i].at['Digestion Tube']],2)
        pipette_left.mix(3,plasmid.loc[i].at['Buffer'],pcr[plasmid.loc[i].at['Digestion Tube']])
        pipette_left.blow_out()
        pipette_left.drop_tip()  


# pick up PME1 -> dispense into pcr tube -> get rid of tip

    for i, row in plasmid.iterrows():
        pipette_left.pick_up_tip()
        pipette_left.aspirate(plasmid.loc[i].at['PME1'],cold_tuberack['A4'],2)
        pipette_left.dispense(plasmid.loc[i].at['PME1'],pcr[plasmid.loc[i].at['Digestion Tube']],2)
        pipette_left.mix(3,plasmid.loc[i].at['PME1'],pcr[plasmid.loc[i].at['Digestion Tube']])
        pipette_left.blow_out()
        pipette_left.drop_tip()


    for i, row in plasmid.iterrows():

        if plasmid.loc[i].at['total volume'] > float('10'):
            pipette_right.pick_up_tip()
            pipette_right.mix(3,plasmid.loc[i].at['total volume'],pcr[plasmid.loc[i].at['Digestion Tube']])
            pipette_right.drop_tip()
        else:
            pipette_left.pick_up_tip()
            pipette_left.mix(3,plasmid.loc[i].at['total volume'],pcr[plasmid.loc[i].at['Digestion Tube']])
            pipette_left.drop_tip()

 #mixes contents around using the pipette tip  (reps,max volume,location)


    thermo.close_lid()
    thermo.set_lid_temperature(105)
    thermo.set_block_temperature(37,0,30, block_max_volume = 44) #temp,seconds,minutes,ramprate(danger),max vol
    thermo.set_block_temperature(65,0,20, block_max_volume = 44)
    thermo.set_block_temperature(4, block_max_volume = 44)

    protocol.pause('wait until its time to dispense the product')

    thermo.open_lid()

    for i, row in plasmid.iterrows():

        if plasmid.loc[i].at['total volume'] > float('10'):
            pipette_right.pick_up_tip()
            pipette_right.aspirate(plasmid.loc[i].at['total volume'],pcr[plasmid.loc[i].at['Digestion Tube']],2)
            pipette_right.dispense(plasmid.loc[i].at['total volume'],finalprodtubes[plasmid.loc[i].at['final tube']],2)
            pipette_right.blow_out()
            pipette_right.drop_tip()
        
        else:
            pipette_left.pick_up_tip()
            pipette_left.aspirate(plasmid.loc[i].at['total volume'],pcr[plasmid.loc[i].at['Digestion Tube']],2)
            pipette_left.dispense(plasmid.loc[i].at['total volume'],finalprodtubes[plasmid.loc[i].at['final tube']],2)
            pipette_left.blow_out()
            pipette_left.drop_tip()

    thermo.close_lid()

    #could add a purple loading dye mix step here.

